## 4개월 간 s-마트의 영수증 데이터를 이용하여 마케팅 전략을 수립하려 한다.

s_mart.csv

|컬럼|정의|type|
|:---|:---|:---:|
|goods|상품 구매 수|int64|
|purchases|결제 총 금액|float64|
|parking|주차 여부 (값이 0이면 주차하지 않음, 1이면 주차하였음)|int64|
|rental_bag|장바구니 대여 여부(값이 0이면 대여하지 않음, 1이면 소형 사이즈 대여, 2면 대형 사이즈 대여)|int64|
|date|결제 날짜|object|  

정답 및 해설 : https://tjd229.tistory.com/21

In [2]:
import numpy as np
import pandas as pd

df = pd.read_csv("../Dataset/s_mart.csv")
df.shape

(8043, 5)

### Q1. 여러 변수간 상관관계를 확인하고자 한다. 상품 구매 수(goods), 결제 총 금액(purchases), 주차 여부(parking), 장바구니 대여 여부(rental_bag)간 피어슨 상관분석을 수행하고 그 결과 중 상관계수 절대값이 가장 큰 값을 구하시오
※ 결측값이 포함된 데이터는 제거한 후에 상관분석을 수행하시오  
※ 정답은 반올림하여 소수점 첫째 자리까지 출력하시오.
(정답 예시: 22.9)

In [9]:
df1 = df.copy()


df1_1 = df1.dropna()
df1_1.shape

df1_1.drop('date', axis=1).corr(method='pearson')

round(0.466990,1)

0.5

### Q2. 특정 날짜 사이의 결제 총 금액(purchases) 평균을 구하려고 한다. 다음 단계에 따라 분석을 수행하고 질문에 답하시오.

단계 1 : 결측값(Null)이 포함된 모든 행을 제거하시오  
단계 2 : 단계 1을 수행한 데이터에서 결제 총 금액(purchases)이 49710인 날짜 st, 결제 총 금액(purchases)이 96120인 날짜 ed를 구한다.  
- st와 ed 조건을 만족하는 데이터는 유일하다.

단계 3 : 단계 2에서 구한 st와 ed 날짜 사이의 결제 총 금액(purchases) 평균을 구하시오(st와 ed 날도 포함한다.)  
  
※ 정답은 반올림하여 소수점 셋째 자리까지 출력하시오.
(정답 예시: 0.229)

In [31]:
df2 = df.copy()
df2_1 = df2.dropna()

df2_1['date'] = pd.to_datetime(df2_1['date'])
# df2_1['date'].loc[:,'date'] = pd.to_datetime(df2_1['date']) 

st = df2_1[df2_1['purchases'] == 49710]['date'].values[0]
ed = df2_1[df2_1['purchases'] == 96120]['date'].values[0]
st, ed

# df2_1.info()
mask = (df2_1['date']>=st) & (df2_1['date']<=ed)
print(round(df2_1.loc[mask,'purchases'].mean(),3))


155362.209


/tmp/ipykernel_43693/4055509465.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_1['date'] = pd.to_datetime(df2_1['date'])


### Q3. 데이터를 바탕으로 장바구니 대여 여부(rental_bag)를 예측하는 모델을 만드려고 한다. 다음 단계에 따라 분석을 수행하고 질문에 답하시오.

단계 1 : 결제 총 금액(purchases)의 결측 값(Null)을 결제 총 금액(purchases)의 평균값으로 대체한다.  
단계 2 : 장바구니 대여 여부(rental_bag) 컬럼 값을 대여하지 않은 경우는 0, 대여한 경우는 종류에 상관없이 1로 변환한다.  
단계 3 : 날짜가 2월이 아닌 데이터를 Train Set으로, 2월인 데이터를 Test Set으로 분할한다.  
단계 4 : 상품 구매 수(goods)와 결제 총 금액(purchases) 컬럼에 대해 Train Set을 기준으로 Z-score 표준화(Standardization) 한다.  
단계 5 : Train Set으로 아래 조건에 따라 로지스틱 회귀분석을 학습한다.  
- 독립 변수 : 상품 구매 수(goods), 결제 총 금액(purchases), 주차 여부(parking)  
- 종속 변수 : 장바구니 대여 여부(rental_bag)  

단계 6 : 단계 5에서 학습한 모델을 Test Set에 적용하여 장바구니 대여 여부(rental_bag)를 예측한다. Test Set에 주어진 종속 변수를 활용하여 F1-score로 모델을 평가하시오  
            
※ 정답은 반올림하여 소수점 둘째 자리까지 출력하시오.  
(정답 예시: 2.29)

In [49]:

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score

df3 = df.copy()

# m = df3['purchases'].mean()
# df3.loc[df3['purchases'].isnull(), 'purchases'] = m
df3['purchases'] = df3['purchases'].fillna(df3['purchases'].mean())

# df3['rental_bag'].unique()
df3['rental_bag'] = df3['rental_bag'].apply(lambda x: 0 if x == 0 else 1)

df3['date'] = pd.to_datetime(df3['date'])

print(df3['date'].dt.month.unique())
train_st = df3[df3['date'].dt.month != 2 ]
test_st  = df3[df3['date'].dt.month == 2 ]

train_st.shape, test_st.shape

scaler = StandardScaler()
scaler.fit(X = train_st.loc[:,'goods':'purchases'])
train_st.loc[:,'goods':'purchases'] = scaler.transform(X = train_st.loc[:,'goods':'purchases'])

model = LogisticRegression(random_state=229)
model.fit(X = train_st[['goods','purchases','parking']], y = train_st['rental_bag'])

test_st.loc[:,'goods':'purchases'] = scaler.transform(X = test_st.loc[:,'goods':'purchases'])

pred = model.predict(X = test_st[['goods','purchases','parking']])
# yhat = model.predict(X=test_st.drop('rental_bag', axis=1))

round(f1_score(test_st['rental_bag'], pred),2)

[ 2 11 12  1]


0.82

#### 필요 라이브러리 함수,클래스 및 설정값 목록  


from sklearn.linear_model import LogisticRegression  
from sklearn.preprocessing import StandardScaler  
from sklearn.metrics import f1_score  
random_state = 229  
문제 지시 외 Default 값 사용  